<a href="https://colab.research.google.com/github/g7199/DL-HW-CODES/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from seq2seq import Seq2seq
from peeky_seq2seq import PeekySeq2seq


# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()

# 입력 반전 여부 설정 =============================================
is_reverse = True
if is_reverse:
    x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]
# ================================================================

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 5
hideen_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# 일반 혹은 엿보기(Peeky) 설정 =====================================
# model = Seq2seq(vocab_size, wordvec_size, hideen_size)
model = PeekySeq2seq(vocab_size, wordvec_size, hideen_size)
# ================================================================
optimizer = Adam()
trainer = Trainer(model, optimizer)

peeky_acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
                batch_size=batch_size, max_grad=max_grad, eval_interval=150)

    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                    id_to_char, verbose, is_reverse)

    acc = float(correct_num) / len(x_test)
    peeky_acc_list.append(acc)
    print('검증 정확도 %.3f%%' % (acc * 100))

# 그래프 그리기
x_peeky = np.arange(len(peeky_acc_list))
plt.plot(x_reverse, peeky_acc_list, marker='o')
plt.xlabel('에폭')
plt.ylabel('정확도')
plt.ylim(0, 1.0)
plt.show()

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 4.03
| 에폭 1 |  반복 151 / 351 | 시간 8[s] | 손실 2.20
| 에폭 1 |  반복 301 / 351 | 시간 16[s] | 손실 1.80
Q 77+85  
T 162 
☒ 102 
---
Q 975+164
T 1139
☒ 1122
---
Q 582+84 
T 666 
☒ 102 
---
Q 8+155  
T 163 
☒ 102 
---
Q 367+55 
T 422 
☒ 102 
---
Q 600+257
T 857 
☒ 1122
---
Q 761+292
T 1053
☒ 1122
---
Q 830+597
T 1427
☒ 1111
---
Q 26+838 
T 864 
☒ 1023
---
Q 143+93 
T 236 
☒ 102 
---


KeyboardInterrupt: ignored

In [1]:
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [21]:
num_samples = 30000

In [4]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [5]:
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())
    return sent

In [6]:
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent).encode('utf-8'))

have you had dinner?
b'avez-vous deja dine?'


In [28]:
cd /content/

/content


In [29]:
with open("fra_data.txt", "w") as f:
  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      enc, dec, dum = line.split('\t')
      enc = preprocess_sentence(enc)[0:7].ljust(8)
      dec = preprocess_sentence(dec)[0:7].ljust(7)
      dec = '_'+dec 
      data = enc + dec + '\n'
      f.write(data)
      if i == num_samples - 1:
        break

In [30]:
!mv fra_data.txt /content/drive/MyDrive/dl/DLFromScratch2/dataset

In [31]:
cd /content/drive/MyDrive/dl/DLFromScratch2/Chap07-Seq2Seq/

/content/drive/MyDrive/dl/DLFromScratch2/Chap07-Seq2Seq


In [24]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from seq2seq import Seq2seq
from peeky_seq2seq import PeekySeq2seq


# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('fra_data.txt')
char_to_id, id_to_char = sequence.get_vocab()

# 입력 반전 여부 설정 =============================================
is_reverse = True
if is_reverse:
    x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]
# ================================================================

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 5
hideen_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# 일반 혹은 엿보기(Peeky) 설정 =====================================
# model = Seq2seq(vocab_size, wordvec_size, hideen_size)
model = PeekySeq2seq(vocab_size, wordvec_size, hideen_size)
# ================================================================
optimizer = Adam()
trainer = Trainer(model, optimizer)

peeky_acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
                batch_size=batch_size, max_grad=max_grad, eval_interval=150)

    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                    id_to_char, verbose, is_reverse)

    acc = float(correct_num) / len(x_test)
    peeky_acc_list.append(acc)
    print('검증 정확도 %.3f%%' % (acc * 100))

# 그래프 그리기
x_peeky = np.arange(len(peeky_acc_list))
plt.plot(x_reverse, peeky_acc_list, marker='o')
plt.xlabel('에폭')
plt.ylabel('정확도')
plt.ylim(0, 1.0)
plt.show()

| 에폭 1 |  반복 1 / 210 | 시간 0[s] | 손실 4.03
| 에폭 1 |  반복 151 / 210 | 시간 11[s] | 손실 2.82
Q i reste 
T je me s
☒ je s e 
---
Q when ar 
T quand e
☒ jo s e 
---
Q you're  
T tu es t
☒ jo s e 
---
Q did the 
T ont-ils
☒ je s e 
---
Q tom is  
T tom est
☒ jo s e 
---
Q he bloc 
T il m'a 
☒ je s e 
---
Q i'm goi 
T je vais
☒ je s e 
---
Q can we  
T pouvons
☒ jo s e 
---
Q wind th 
T remonte
☒ jo s e 
---
Q are you 
T etes-vo
☒ to s e 
---
검증 정확도 0.000%
| 에폭 2 |  반복 1 / 210 | 시간 0[s] | 손실 2.34
| 에폭 2 |  반복 151 / 210 | 시간 10[s] | 손실 2.01
Q i reste 
T je me s
☒ je se s
---
Q when ar 
T quand e
☒ nous no
---
Q you're  
T tu es t
☒ tu es e
---
Q did the 
T ont-ils
☒ je suis
---
Q tom is  
T tom est
☑ tom est
---
Q he bloc 
T il m'a 
☒ il es e
---
Q i'm goi 
T je vais
☒ je suis
---
Q can we  
T pouvons
☒ ne es e
---
Q wind th 
T remonte
☒ nous es
---
Q are you 
T etes-vo
☒ es-es e
---
검증 정확도 6.600%
| 에폭 3 |  반복 1 / 210 | 시간 0[s] | 손실 1.66
| 에폭 3 |  반복 151 / 210 | 시간 12[s] | 손실 1.51
Q i reste 
T je m

NameError: ignored